# Seattle Airbnbn Listings Analysis_CRISP-DM
- __Author: Fan Yuan__
- __Created: 07/02/2019__

## Context
Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in Seattle, WA. This project is going to analyse the Seattle Airbnb Listings to see if any insight from data could help Airbnb optimize the business.

In this project, __CRISP-DM (Cross-Industry Standard Process for Data Mining)__ will be applied to better frame the problem and get the insight from data analysis.

- __CRISP-DM__:
    * Business Understanding
    * Data Understanding
    * Data Preparation
    * Modeling
    * Evaluation
    * Deployment


## To analyse:
This project will analyse the Seattle Airbnbn Listings as following:

- Describe the vibe of each Seattle neighborhood using listing descriptions, is there any key factors which largely affect the listing price
    * Explore if there's any common characteristics among Seattle beighborhood, and it might be opportunities for Airbnb to get insight of what kind of neighbourhood are more likely to host Airbnb
    * Probably also look at if there's any preference on neighbourhood among visitors
    
    
- Find the busiest times of the year to visit Seattle and by how much prices spike
    * For Airbnb, it'll be a loss if the available room can't satisfy the user demand within busy season
    * Also, the scale of price increase will be helpful for business opportunities
    
    
- Varify that if there's a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle
    * Get an insight on how the host and vistors change in period
    
    
- What factor relates to higher guests review most?
    * If we could get an idea of which kind of hosts and room are more likely to get a higher rivew from guests, as Airbnb, they can recommend or advice the hosts to get better review next time

## Load data & take an initial view

First, load in dataset and have a cursory overview to understand the data.

Three data sets are provided:
- listings: including full descriptions and average review score
- calendar: including listing id and the price and availability for that day
- reviews: including unique id for each reviewer and detailed comments

In [92]:
# Import libraries necessary 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import copy

# Pretty display for notebooks
%matplotlib inline
matplotlib.style.use('ggplot')

In [93]:
# Load the listings, calendar, reviews data
listings = pd.read_csv('seattle/listings.csv')

# Display the first several rows of dataset
display(listings.head())
# display(listings.shape)
display(listings.info())

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url                      3498

None

### Do an initial categorization on features in listings data set:

- DESCRIPTIONS - name, summary, space, description
- HOST - host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost
- HOME DETAILS - property_type, room_type, accomodates, bathrooms, bedrooms, beds, bed_type, amenities
- LISTING ELEMENTS - price, cleaning_fee, extra_people, minimum_nights, maximum_nights, instant_bookable, cancellation policy
- OUTCOME VARIABLES - 'review_scores_rating', 'review_scores_accuracy', review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value'

In [94]:
# Load calendar data
calendar = pd.read_csv('seattle/calendar.csv')
display(calendar.head())
display(calendar.info())

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null object
price         934542 non-null object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB


None

In [95]:
# Check if all 'f' available have 'NaN' in price
# calendar.price[calendar['available']=='f'].isna().mean()
calendar_available_check = calendar.groupby('available')['price'].count().reset_index()
calendar_available_check

,available,price
0,f,0
1,t,934542


It shows all 'f' available in calendar dataset will have NaN in price column

In [125]:
# Check if each listing_id has same rows of availability entries
calendar_date_check = calendar.groupby('listing_id')['date'].count().reset_index()
display(calendar_date_check.head())
calendar_date_check['date'].value_counts()

,listing_id,date
0,3335,365
1,4291,365
2,5682,365
3,6606,365
4,7369,365


365    3818
Name: date, dtype: int64

From the result above, it shows that for each of 3818 listing_id there are 1-year (365) calendar price. This can be utiliszed later to investigate if there's upward or downward trend within the year period.

In [97]:
# Load reviews data
reviews = pd.read_csv('seattle/reviews.csv')
display(reviews.head())
display(reviews.info())

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 6 columns):
listing_id       84849 non-null int64
id               84849 non-null int64
date             84849 non-null object
reviewer_id      84849 non-null int64
reviewer_name    84849 non-null object
comments         84831 non-null object
dtypes: int64(3), object(3)
memory usage: 3.9+ MB


None

## Clean the data for further analysis

### Listings data
Clean up the useless columns from 92 fields to ones that are valuable and helpful for later anlaysis and prediction.


In [98]:
# Check the null percentage to see if need to abandon any columns
listing_null = listings.isnull().sum()/listings.shape[0]
listing_null[listing_null > 0].sort_values(ascending=False).head(20)

license                        1.000000
square_feet                    0.974594
monthly_price                  0.602672
security_deposit               0.511262
weekly_price                   0.473808
notes                          0.420639
neighborhood_overview          0.270299
cleaning_fee                   0.269775
transit                        0.244631
host_about                     0.224987
host_acceptance_rate           0.202462
review_scores_accuracy         0.172342
review_scores_checkin          0.172342
review_scores_value            0.171818
review_scores_location         0.171556
review_scores_cleanliness      0.171032
review_scores_communication    0.170508
review_scores_rating           0.169460
reviews_per_month              0.164222
first_review                   0.164222
dtype: float64

In order to get an idea of how is the 

In [160]:
listings.neighbourhood.value_counts()

Capitol Hill                 351
Ballard                      213
Belltown                     204
Minor                        192
Queen Anne                   187
Fremont                      148
Wallingford                  143
University District          107
First Hill                   105
Stevens                       98
North Beacon Hill             95
Lower Queen Anne              83
Central Business District     81
Greenwood                     79
Columbia City                 61
Ravenna                       59
Phinney Ridge                 56
Magnolia                      55
Green Lake                    52
Atlantic                      50
North Admiral                 48
Mount Baker                   46
Leschi                        44
Eastlake                      41
Maple Leaf                    41
Madrona                       40
Pike Place Market             39
The Junction                  36
Seward Park                   32
Bryant                        32
          

In [193]:
# Describe the vibe of each Seattle neighborhood using listing descriptions
listings[listings['neighbourhood']== 'Queen Anne']['neighborhood_overview']

0                                                     NaN
1       Queen Anne is a wonderful, truly functional vi...
2       Upper Queen Anne is a charming neighborhood fu...
3                                                     NaN
4       We are in the beautiful neighborhood of Queen ...
5       This part of Queen Anne has wonderful views an...
6       Close restaurants, coffee shops and grocery st...
7       We are centrally located between Downtown and ...
8       Walking Score: 92 4 blocks from Kerry Park Fam...
9       Queen Anne is a wonderful, truly functional vi...
10      Queen Anne Hill is a wonderful and historic ar...
11      I am located in the Upper Queen Anne neighborh...
13      Queen Anne hill became a popular spot for the ...
14      Queen Anne is THE BEST and most desirable neig...
16      Lower Queen Anne is amazing - you can walk to ...
17      We are located in Upper Queen Anne in walking ...
18                                                    NaN
19      Upper 

### Calendar data

In order to plot the average price within a year to display how Airbnb price move, here need to:
- Remove the '$' in price column to make it correctly formatted for calculation
- Drop the NaN in price column
- Transfer the date format for better plot.

In [99]:
# Replace the '$' and ',' in price column
calendar['price'] = calendar['price'].str.replace('$','')
calendar['price'] = calendar['price'].str.replace(',','').astype('float64')

In [120]:
calendar_day_mean = calendar.groupby('date')[['price']].mean()
calendar_day_mean.head()

,price
date,
2016-01-04,122.085879
2016-01-05,120.681307
2016-01-06,120.928258
2016-01-07,120.958896
2016-01-08,127.640853


In [101]:
import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode, download_plotlyjs

In [121]:
offline.init_notebook_mode(connected=True)

offline.iplot({
    "data": [go.Scatter(x=calendar_day_mean.index, y=calendar_day_mean.price)],
    "layout": go.Layout(title='Price Trend by Date',
                           xaxis=dict(title='Date'),
                           yaxis=dict(title='Avg'))
})

Obivously, there's a slow but continous __upward trend__ of Seattle Airbnb price from Jan to Jul where reaching the peak, and then it goes down slowly a little after that. In order to see the trend more clearly in high level, aggregate the data into monthly data.

In [119]:
# Resample calendar data by month
calendar_month_mean = copy.copy(calendar)

calendar_month_mean['date'] = pd.to_datetime(calendar_month_mean['date'])
calendar_month_mean = calendar_month_mean.set_index("date")

calendar_month_mean = pd.DataFrame(calendar_month_mean['price'].resample('M').mean())
calendar_month_mean

,price
date,
2016-01-31,121.568409
2016-02-29,124.293927
2016-03-31,128.644488
2016-04-30,135.097005
2016-05-31,139.538183
2016-06-30,147.473137
2016-07-31,152.094150
2016-08-31,150.656594
2016-09-30,143.255949


In [123]:
offline.iplot({
    "data": [go.Scatter(x=calendar_month_mean.index, y=calendar_month_mean.price)],
    "layout": go.Layout(title='Price Trend by Month',
                           xaxis=dict(title='Month'),
                           yaxis=dict(title='Avg'))
})

From the plot above, it clearly shows a peak in Jul with an average of $152. Compared to the lowest price given the period data, it increases about $20 in average.

So we could check the availability of room by time using caleandar data to see if the number of available is low when the price goes upward. Also we could find when the busiest time of vistors going to Seattle by counting how many rooms not available along with time.

In [158]:
calendar_month_available = calendar.groupby('date')['price'].count().reset_index()
calendar_month_available.columns = ['date', 'available_room']

calendar_month_available['date'] = pd.to_datetime(calendar_month_available['date'])
calendar_month_available = calendar_month_available.set_index('date')
calendar_month_available = calendar_month_available.resample('M')[['available_room']].mean()

calendar_month_available

,available_room
date,
2016-01-31,2115.678571
2016-02-29,2528.310345
2016-03-31,2707.677419
2016-04-30,2534.566667
2016-05-31,2579.709677
2016-06-30,2574.800000
2016-07-31,2394.258065
2016-08-31,2462.806452
2016-09-30,2574.866667


In [159]:
offline.iplot({
    "data": [go.Scatter(x=calendar_month_available.index, y=calendar_month_available.available_room)],
    "layout": go.Layout(title='Available room number by Month',
                           xaxis=dict(title='Month'),
                           yaxis=dict(title='Avg'))
})

Combined with the plot of price trend analysis, obviously, the busiest season locates in Jul since the available room number gets the lowest point and the room price also raise to the highest point.

### Review data

id and reviewer_name columns in review data set are not very helpful for analysis actually. Here use review count number along with time to see if there's an upward trend in total visitors to Seattle


In [183]:
reviews_count = copy.copy(reviews)
reviews_count['date'] = pd.to_datetime(reviews_count['date'])

reviews_count = reviews_count.set_index('date')
reviews_count = reviews_count.resample('Q')[['id']].count()
reviews_count

,id
date,
2009-06-30,2
2009-09-30,4
2009-12-31,11
2010-03-31,13
2010-06-30,37
2010-09-30,110
2010-12-31,94
2011-03-31,88
2011-06-30,212


In [182]:
offline.iplot({
    "data": [go.Scatter(x=reviews_count.index, y=reviews_count.id)],
    "layout": go.Layout(title='Review number by Quater',
                           xaxis=dict(title='Quater'),
                           yaxis=dict(title='Review Num'))
})

The plot above clearly shows that there's a general upward trend of total Airbnb visitors to Seattle reaching the peak around 2015 summer.